In [1]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

### Pulling down the Listing Data

In [3]:
r = requests.get('https://www.propertypro.ng/property-for-rent/flat-apartment/?search=&bedroom=&min_price=&max_price=3000000')

soup = BeautifulSoup(r.content, 'html5lib')

In [4]:
soup

<!DOCTYPE html>
<html lang="en"><head>
<title>Flat Apartment for rent  in Nigeria (16,050 listings) | PropertyPro.ng</title>

<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>

<link href="/assets/image/42484569b600758112d45bad3d876f78-favicon.ico" rel="icon" type="image/ico"/>
<meta content="Flat apartment  to let in  Nigeria, Flat Apartment for rent  in  Nigeria" name="description"/>
<meta content="Flat Apartment, Flat apartment for  let in  Nigeria, Flat Apartment for rent  in  Nigeria,  Nigeria Flats, houses, Apartments, land, commercial property, office space, self contain" name="keywords"/>
<meta content="Flat Apartment for rent  in Nigeria" property="og:title"/>
<meta content="Flat apartment  to let in  Nigeria, Flat Apartment for rent  in  Nigeria" property="og:description"/>
<link href="https://www.propertypro.ng/property-for-rent/flat-apartment" rel="canonical"/>

<link href="https://fonts.googleapis.com/css?family=Roboto:300,400,50

In [3]:
listing_divs = soup.select('div[class=single-room-sale\ listings-property]')
print("Number of apartment listings on a page:", len(listing_divs))

Number of apartment listings on a page: 20


In [4]:
listing_divs[0]

<div class="single-room-sale listings-property">
<div class="single-room-img">
<a class="single-room-img-a" href="/property/3-bedroom-flat-apartment-for-rent-ikota-lekki-lagos-1ECXR"> <img alt="3 bedroom Flat / Apartment for rent Ikota Lekki Lagos" class="listings-img" src="https://images.propertypro.ng/medium/e7exz-premium-3-bedroom-apartment-1FN5ThN4XD5FYU2zFUZl.jpg" title="3 bedroom Flat / Apartment for rent Ikota Lekki Lagos"/> </a>
<div class="save-heart">
<a href="/profile/add-favorite/575286"><i class="fa fa-heart"></i></a>
</div>
<div class="room-icons">
<span><span>37</span> <img alt="icon" src="/assets/assets/img/property/2f8b655773948c43f77e8c928e94bbe3-img.png"/> </span>
</div>

</div>
<div class="single-room-text">
<a href="/property/3-bedroom-flat-apartment-for-rent-ikota-lekki-lagos-1ECXR"> <h2 class="listings-property-title">3 BEDROOM FLAT / APARTMENT FOR RENT</h2> </a>
<h4><img alt="icon" src="/assets/assets/img/resultpage/a6b70a0c1c4423f60780bdda189b91e3-location.svg"

In [5]:
listing_divs[0].select('h4')[0].text

'Ikota Lekki Lagos'

In [6]:
listing_divs[0].select('h3[class*=listings-price]')[0].text.strip()

'₦ 2,800,000'

In [7]:
listing_divs[0].select('div[class*=fur-areea]')[0].text.strip().split('\n')

['3 beds', '3 baths', '4 Toilets']

In [8]:
listing_divs[7].select('div[class*=furnished-btn]')[0].text.strip().replace('\n', ' ')

'Serviced Newly Built'

In [9]:
listing_divs[7].select('div[class*=result-list-details]')[0].p.text.strip().replace('Read more', '').replace('FOR RENT: ', '')

'3 bedroom Flat / Apartment for rent Old Ikoyi Ikoyi Lagos.. '

**Now that I've gotten all the relevant information, I'll put all the codes together;**

In [10]:
all_listings_data = []
for indv_listing in listing_divs:
    address = indv_listing.select('h4')[0].text
    price = indv_listing.select('h3[class*=listings-price]')[0].text.strip()
    listing_data = [address, price]
    utilities = indv_listing.select('div[class*=fur-areea]')[0].text.strip().split('\n')
    listing_data.extend(utilities)
    description = indv_listing.select('div[class*=result-list-details]')[0].p.text.strip().replace('Read more', '').replace('FOR RENT: ', '')
    extra = indv_listing.select('div[class*=furnished-btn]')[0].text.strip().replace('\n', ' ')
    listing_data.append(extra + description)
    all_listings_data.append(listing_data)
    
pd.DataFrame(all_listings_data)

,0,1,2,3,4,5
0,Ikota Lekki Lagos,"₦ 2,800,000",3 beds,3 baths,4 Toilets,Newly Built3 bedroom Flat / Apartment for rent...
1,OLALEYE Estate Alaka Iponri Surulere Lagos,"₦ 2,500,000",3 beds,4 baths,4 Toilets,3 bedroom Flat / Apartment for rent Old Ikoyi ...
2,"Off Kudirat Abiola way, Julie EstateOregun Ike...","₦ 2,500,000",3 beds,3 baths,4 Toilets,3 bedroom Flat / Apartment for rent Old Ikoyi ...
3,Ikate Lekki Lagos,"₦ 3,000,000",3 beds,4 baths,4 Toilets,Serviced3 bedroom Flat / Apartment for rent Ol...
4,Lekki Phase 1 Lekki Lagos,"₦ 2,800,000",3 beds,3 baths,4 Toilets,3 bedroom Flat / Apartment for rent Old Ikoyi ...
5,ShimawaMowe Obafemi Owode Ogun,"₦ 260,000",3 beds,1 baths,2 Toilets,Newly Built3 bedroom Flat / Apartment for rent...
6,Lekki Phase 1 Lekki Lagos,"₦ 3,000,000",3 beds,3 baths,4 Toilets,3 bedroom Flat / Apartment for rent Old Ikoyi ...
7,-Old Ikoyi Ikoyi Lagos,"$ 40,000",3 beds,3 baths,4 Toilets,Serviced Newly Built3 bedroom Flat / Apartment...
8,Oniru Victoria Island Lagos,"₦ 3,000,000",3 beds,3 baths,4 Toilets,Serviced3 bedroom Flat / Apartment for rent Ol...
9,Ologolo Lekki Lagos,"₦ 2,600,000",3 beds,3 baths,4 Toilets,Serviced Newly Built3 bedroom Flat / Apartment...


**Writing a dynamic function to get the data;**

In [11]:
def parse_listing_data(location, max_price, rows_of_data=2000):
    all_listings_data = []
    page_num = 0
    
    while page_num < (rows_of_data / 20):
        url = "https://www.propertypro.ng/property-for-rent/flat-apartment/?search=" + location \
              + "&bedroom=&min_price=&max_price=" + str(max_price) + "&page=" + str(page_num)
        r = requests.get(url)
        soup = BeautifulSoup(r.content, 'html5lib')
        listing_divs = soup.select('div[class=single-room-sale\ listings-property]')
        if len(listing_divs) == 0:
            break 
            
        for indv_listing in listing_divs:
            address = indv_listing.select('h4')[0].text
            price = indv_listing.select('h3[class*=listings-price]')[0].text.strip()
            listing_data = [address, price]
            utilities = indv_listing.select('div[class*=fur-areea]')[0].text.strip().split('\n')
            listing_data.extend(utilities)
            description = indv_listing.select('div[class*=result-list-details]')[0].p.text.strip().replace('Read more', '').replace('FOR RENT: ', '')
            extra = indv_listing.select('div[class*=furnished-btn]')[0].text.strip().replace('\n', ' ')
            listing_data.append(extra + description)
            all_listings_data.append(listing_data)
        page_num += 1    
    listing_df = pd.DataFrame(all_listings_data, columns=['neighborhood', 'price', 'beds', 'baths', 'toilets', 'extra'])
    
    listing_df.to_csv("rent_listings_"+location+".csv", index=False)
    return listing_df

### Getting Lagos' Listing Data

In [12]:
lagos_data = parse_listing_data('lagos', 2500000, 5000)
lagos_data

,neighborhood,price,beds,baths,toilets,extra
0,OLALEYE Estate Alaka Iponri Surulere Lagos,"₦ 2,500,000",3 beds,4 baths,4 Toilets,Brand new service 3 bedroom flat all rooms ens...
1,"Off Kudirat Abiola way, Julie EstateOregun Ike...","₦ 2,500,000",3 beds,3 baths,4 Toilets,The property is a 3 Bedrooms Upscale FLAT all ...
2,-Old Ikoyi Ikoyi Lagos,"$ 40,000",3 beds,3 baths,4 Toilets,Serviced Newly Built3 bedroom Flat / Apartment...
3,...Lekki Phase 1 Lekki Lagos,"₦ 800,000",1 beds,1 baths,1 Toilets,"For more information, Pricing, and Private Vie..."
4,Lekki Phase 1 Lekki Lagos,"₦ 1,200,000",1 beds,1 baths,1 Toilets,"For more information, Pricing, and Private Vie..."
...,...,...,...,...,...,...
4995,Peace EstateApple Junction Amuwo Odofin Lagos,"₦ 1,200,000",2 beds,2 baths,3 Toilets,Just 3 occupants..
4996,Moore roadYaba Lagos,"₦ 1,000,000",2 beds,2 baths,3 Toilets,FurnishedSpacious 3 bedroom 3t2b one ensuit wi...
4997,SPGOlogolo Lekki Lagos,"₦ 1,500,000",2 beds,2 baths,3 Toilets,Very nice 2bedroom apartment at spg free from ...
4998,Lekki Phase 1 Lekki Lagos,"₦ 1,600,000",2 beds,2 baths,2 Toilets,For Rent Lekki phase 1 Luxury 2bedroom apartme...


### Getting Ibadan's Data

In [13]:
ibadan_data = parse_listing_data('oyo', 3000000, 2000)
ibadan_data

,neighborhood,price,beds,baths,toilets,extra
0,Iyaganku GRA quarters IbadanIyanganku Ibadan Oyo,"₦ 1,000,000",3 beds,3 baths,3 Toilets,*To let Three bedroom flat en suit is availabl...
1,Behind DSS ile tuntun nihort Jericho ibadanIdi...,"₦ 450,000",3 beds,3 baths,3 Toilets,"*To let* 4 flat of 3 bedroom with 3 toilets, b..."
2,Yidi apete junction ibadan Ibadan Polytechnic...,"₦ 126,000",1 beds,1 baths,1 Toilets,Serviced Newly BuiltA room selfcon at Yidi ape...
3,"Adeoyo GRA, off ring road, ibadan Ring Rd Ibad...","₦ 750,000",3 beds,3 baths,4 Toilets,Newly BuiltFinely finished 3 bedroom Apartment...
4,"Opposition nnpc filling station, akala express...","₦ 600,000",3 beds,3 baths,4 Toilets,Newly BuiltBrand new 3 bedroom Apartment for r...
...,...,...,...,...,...,...
507,"Salute, tipper. Akala express, ibadanAkala Exp...","₦ 200,000",1 beds,1 baths,2 Toilets,Newly BuiltVery neat newly built self contain ...
508,OrogunIbadan Oyo,"₦ 180,000",3 beds,2 baths,2 Toilets,Renovated Well water with pump Wardrobe Store ..
509,Elebu OjaAkala Express Ibadan Oyo,"₦ 350,000",2 beds,2 baths,2 Toilets,Furnished Newly Built2 bedroom Shared Apartmen...
510,Onigbodogi Apete IbadanAjibode Ibadan Oyo,"₦ 120,000",1 beds,1 baths,1 Toilets,A very neat room self contain with modern faci...


### Getting Abuja's Data

In [14]:
abuja_data = parse_listing_data('abuja', 4000000, 3000)
abuja_data

,neighborhood,price,beds,baths,toilets,extra
0,Sam najuma estate GaladimawaLokogoma Abuja,"₦ 1,300,000",3 beds,3 baths,4 Toilets,Sharp 2nd floor 3 bedroom apartment @ Dr Sam n...
1,DawakiGwarinpa Abuja,"₦ 1,500,000",3 beds,3 baths,4 Toilets,Newly BuiltIt's newly built and never been occ...
2,Gaduwa Abuja,"₦ 1,500,000",3 beds,3 baths,4 Toilets,"3bedroom blocks of flat in Gaduwa District, gr..."
3,Maitama Abuja,"₦ 2,500,000",2 beds,2 baths,4 Toilets,ServicedA Serviced 2 bedroom flat in #maitama ...
4,In an estate behind games village Garki 1 Abuja,"₦ 1,500,000",2 beds,2 baths,3 Toilets,"2 Bedroom Flat With CCTV, Constant Water Suppl..."
...,...,...,...,...,...,...
734,by ABC TRANSPORTUtako Abuja,"₦ 3,500,000",3 beds,3 baths,4 Toilets,Furnished Serviced Newly BuiltTastefully finis...
735,...Durumi Abuja,"₦ 2,800,000",0 beds,0 baths,0 Toilets,Available for yearly rent is an exquisitely fi...
736,...Guzape Abuja,"₦ 4,000,000",0 beds,0 baths,0 Toilets,TO LET... Brand new spacious 4bedrooms flat. T...
737,...Wuye Abuja,"₦ 3,500,000",0 beds,0 baths,0 Toilets,A well spacious renovated 3bedrooms terrace du...


### Getting Ogun's Data

In [15]:
ogun_data = parse_listing_data('ogun', 2000000, 2000)
ogun_data

,neighborhood,price,beds,baths,toilets,extra
0,ShimawaMowe Obafemi Owode Ogun,"₦ 260,000",3 beds,1 baths,2 Toilets,"Newly BuiltA newly built 1 bed, 2 beds and 3 b..."
1,Adigbe Abeokuta Ogun,"₦ 300,000",2 beds,3 baths,3 Toilets,2 bedroom Shared Apartment Flat / Apartment fo...
2,Private EstateMagboro Obafemi Owode Ogun,"₦ 250,000",1 beds,1 baths,2 Toilets,Spacious Rooms Ample Parking Space Kitchen Cab...
3,MagboroMagboro Obafemi Owode Ogun,"₦ 350,000",2 beds,2 baths,2 Toilets,24-hours Security Spacious Rooms Ample Parking...
4,Private EstateArepo Ogun,"₦ 450,000",2 beds,2 baths,3 Toilets,24-hours Security Spacious Rooms Ample Parking...
...,...,...,...,...,...,...
183,"Ima Road, Abiola Way, Abeokuta.Abeokuta Ogun","₦ 250,000",2 beds,2 baths,3 Toilets,"A luxury 2 bedroom flat @ Ima Road, Abiola Way..."
184,Egan road Iyesi otaAdo Odo Ota Ogun,"₦ 180,000",3 beds,4 baths,4 Toilets,Furnished Newly BuiltA very lovely 3bedroom fl...
185,Abeokuta Street Dalemo Sango otaAdalemo Ado Od...,"₦ 200,000",1 beds,1 baths,1 Toilets,A newly well furnished room and parlor mini fl...
186,Ibafo Obafemi Owode Ogun,"₦ 350,000",2 beds,2 baths,2 Toilets,Newly Built2 bedroom apartment in a newly buil...


### Getting PortHarcourt's Data

In [16]:
ph_data = parse_listing_data('rivers', 3000000, 2500)
ph_data

,neighborhood,price,beds,baths,toilets,extra
0,GRA phase 3 tombia extensionNew Gra Port Harco...,"₦ 1,500,000",2 beds,2 baths,2 Toilets,Newly Built2no newly built 2bed room flats in ...
1,Ada George Port Harcourt Rivers,"₦ 500,000",2 beds,2 baths,2 Toilets,Furnished Newly BuiltA VIRGIN TWO BEDROOM FLAT...
2,odili roadTrans Amadi Port Harcourt Rivers,"₦ 550,000",1 beds,1 baths,2 Toilets,One bedroom for [email protected] #550k odili ...
3,Trans Amadi Port Harcourt Rivers,"₦ 650,000",beds,baths,Toilets,Decent King Size 2 Bedroom Flat For Rent In Wo...
4,...Trans Amadi Port Harcourt Rivers,"₦ 800,000",0 beds,0 baths,0 Toilets,King Size 2 Bedroom For Rent In Woji Port Har...
...,...,...,...,...,...,...
100,Emenike St.Eagle Island Rumueme Oroakwo Port H...,"₦ 3,000,000",8 beds,5 baths,5 Toilets,Furnished3 bedroom flat for rent in Eagle lsla...
101,RumuodaraPort Harcourt Rivers,"₦ 800,000",3 beds,4 baths,4 Toilets,3 bedroom Flat / Apartment for rent Rumuodara ...
102,Off peter odili roadTrans Amadi Port Harcourt ...,"₦ 1,400,000",3 beds,4 baths,4 Toilets,Newly BuiltUltra modern 3 bedroom flat situate...
103,"N0.3 DR.ANJU CLOSE, OFF CHIEF UMEH STREET, AGI...","₦ 800,000",3 beds,2 baths,2 Toilets,Tastefully finished 3 bedroom flat in a serene...
